In [ ]:
from ultralytics import YOLO

# YOLOv8 모델 초기화
model = YOLO('yolov8n.pt')  # Nano 모델 사용

# 데이터셋 설정 파일 경로
data_path = r'C:\Users\kj100\Downloads\American Sign Language Letters.v1-v1.yolov8\data.yaml'  # data.yaml 파일 경로 (YOLO 형식)

# 모델 학습
model.train(
    data=data_path,    # 데이터셋 경로
    epochs=10,         # 학습 반복 횟수
    imgsz=640,         # 입력 이미지 크기
    batch=16,          # 배치 크기
    name='asl_yolov8_model'  # 저장 디렉토리 이름
)

# 학습된 모델 저장
model.save('asl_yolov8_model.pt')
print("YOLOv8 모델 학습 완료")


In [1]:
import cv2
import mediapipe as mp
from ultralytics import YOLO

# YOLOv8 모델 로드
model = YOLO('asl_yolov8_model.pt')

# MediaPipe Hands 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 카메라 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

print("카메라가 실행되었습니다. 'q'를 눌러 종료하세요.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 가져올 수 없습니다.")
        break

    # 화면 좌우 반전 (거울 효과)
    frame = cv2.flip(frame, 1)

    # YOLOv8 예측
    results = model.predict(source=frame, conf=0.5, show=False)  # YOLOv8 예측
    detections = results[0].boxes if results[0].boxes else []  # YOLOv8 탐지 결과

    # MediaPipe 손 랜드마크 감지
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    hand_results = hands.process(rgb_frame)

    # MediaPipe 랜드마크 표시
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
            )

    # YOLO 탐지 결과 표시 및 알파벳 출력
    detected_alphabet = None  # 인식된 알파벳 초기화
    for detection in detections:
        coords = detection.xyxy[0].tolist()  # 좌표 정보 리스트로 변환
        x1, y1, x2, y2 = map(int, coords[:4])  # 바운딩 박스 좌표
        conf = detection.conf[0].item()  # 신뢰도
        class_id = int(detection.cls[0].item())  # 클래스 ID

        # 신뢰도 기준 추가
        if conf >= 0.6:  # 신뢰도가 60% 이상인 경우만 처리
            detected_alphabet = model.names[class_id]
            label = f"{detected_alphabet} {conf:.2f}"

            # 바운딩 박스 그리기
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # 콘솔에 인식된 알파벳 출력
    if detected_alphabet:
        print(f"인식된 알파벳: {detected_alphabet}")

    # 화면 출력
    cv2.imshow("YOLOv8 + MediaPipe", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()
hands.close()


카메라가 실행되었습니다. 'q'를 눌러 종료하세요.

0: 480x640 (no detections), 145.0ms
Speed: 9.0ms preprocess, 145.0ms inference, 7.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 94.0ms
Speed: 3.0ms preprocess, 94.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 74.1ms
Speed: 3.0ms preprocess, 74.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 113.1ms
Speed: 1.0ms preprocess, 113.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 85.0ms
Speed: 2.0ms preprocess, 85.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 76.6ms
Speed: 2.0ms preprocess, 76.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 113.0ms
Speed: 1.0ms preprocess, 113.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 122.0

In [6]:
import cv2
import mediapipe as mp
from ultralytics import YOLO

# YOLOv8 모델 로드
model = YOLO('asl_yolov8_model.pt')

# MediaPipe Hands 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 카메라 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

print("카메라가 실행되었습니다. 'q'를 눌러 종료하세요.")

# 단어를 형성하기 위한 변수
current_alphabet = None
word = []

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 가져올 수 없습니다.")
        break

    # 화면 좌우 반전 (거울 효과)
    frame = cv2.flip(frame, 1)

    # YOLOv8 예측
    results = model.predict(source=frame, conf=0.5, show=False, verbose=False)  # YOLOv8 예측 (로그 숨김)
    detections = results[0].boxes if results[0].boxes else []  # YOLOv8 탐지 결과

    # YOLO 탐지 결과 표시
    detected_alphabet = None
    for detection in detections:
        coords = detection.xyxy[0].tolist()  # 바운딩 박스 좌표
        x1, y1, x2, y2 = map(int, coords[:4])
        conf = detection.conf[0].item()
        if conf >= 0.6:  # 신뢰도 기준 추가
            class_id = int(detection.cls[0].item())  # 클래스 ID
            detected_alphabet = model.names[class_id]

            # YOLO 바운딩 박스 및 레이블 표시
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            label = f"{detected_alphabet} {conf:.2f}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # MediaPipe 손 랜드마크 감지
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    hand_results = hands.process(rgb_frame)

    # MediaPipe 랜드마크 표시
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
            )

    # 새 알파벳 감지 시 단어에 추가
    if detected_alphabet and detected_alphabet != current_alphabet:
        current_alphabet = detected_alphabet
        word.append(detected_alphabet)
        print(f"인식된 알파벳: {detected_alphabet}")

        # 단어 완성 여부 확인
        if "".join(word).lower() == "hello":
            print("완성된 단어: HELLO")
            word = []  # 단어를 초기화하여 다시 시작

    # 화면 출력
    cv2.imshow("YOLOv8 + MediaPipe", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()
hands.close()


카메라가 실행되었습니다. 'q'를 눌러 종료하세요.
인식된 알파벳: M
인식된 알파벳: K
인식된 알파벳: I
인식된 알파벳: K
인식된 알파벳: J
인식된 알파벳: Y


In [10]:
import cv2
import mediapipe as mp
from ultralytics import YOLO
import time

# YOLOv8 모델 로드
model = YOLO('asl_yolov8_model.pt')

# MediaPipe Hands 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 카메라 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

print("카메라가 실행되었습니다. 'q'를 눌러 종료하세요.")

# 알파벳 비교를 위한 변수
current_alphabet = None
last_wrong_time = 0  # 틀렸을 때의 타임스탬프
wrong_display_duration = 2  # 틀렸을 때 빨간색 상자를 표시할 시간(초)

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 가져올 수 없습니다.")
        break

    # 화면 좌우 반전 (거울 효과)
    frame = cv2.flip(frame, 1)

    # YOLOv8 예측
    results = model.predict(source=frame, conf=0.5, show=False, verbose=False)  # YOLOv8 예측 (로그 숨김)
    detections = results[0].boxes if results[0].boxes else []  # YOLOv8 탐지 결과

    # YOLO 탐지 결과 표시
    detected_alphabet = None
    for detection in detections:
        coords = detection.xyxy[0].tolist()  # 바운딩 박스 좌표
        x1, y1, x2, y2 = map(int, coords[:4])
        conf = detection.conf[0].item()
        if conf >= 0.6:  # 신뢰도 기준 추가
            class_id = int(detection.cls[0].item())  # 클래스 ID
            detected_alphabet = model.names[class_id]

            # 알파벳 비교 및 처리
            if detected_alphabet == current_alphabet:
                # 동일한 알파벳: 파란색 상자
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                label = f"Detected: {detected_alphabet}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            else:
                # 다른 알파벳: 빨간색 상자
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                label = f"Wrong: {detected_alphabet if detected_alphabet else 'None'}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                last_wrong_time = time.time()  # 틀린 시간 기록

                # 출력문에 추가 정보 표시
                print(f"틀렸습니다! 현재 알파벳: {current_alphabet}, 감지된 알파벳: {detected_alphabet}")

            # 현재 알파벳 업데이트
            current_alphabet = detected_alphabet

    # MediaPipe 손 랜드마크 감지
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    hand_results = hands.process(rgb_frame)

    # MediaPipe 랜드마크 표시
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
            )

    # 틀렸을 때 빨간색 상자 지속 시간 처리
    if time.time() - last_wrong_time < wrong_display_duration:
        cv2.putText(frame, "Wrong Input!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    # 화면 출력
    cv2.imshow("YOLOv8 + MediaPipe", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()
hands.close()


카메라가 실행되었습니다. 'q'를 눌러 종료하세요.
틀렸습니다! 현재 알파벳: None, 감지된 알파벳: F
틀렸습니다! 현재 알파벳: F, 감지된 알파벳: Y
틀렸습니다! 현재 알파벳: Y, 감지된 알파벳: K
틀렸습니다! 현재 알파벳: K, 감지된 알파벳: V
틀렸습니다! 현재 알파벳: V, 감지된 알파벳: K
틀렸습니다! 현재 알파벳: K, 감지된 알파벳: F
틀렸습니다! 현재 알파벳: F, 감지된 알파벳: K
틀렸습니다! 현재 알파벳: K, 감지된 알파벳: F
틀렸습니다! 현재 알파벳: F, 감지된 알파벳: K
틀렸습니다! 현재 알파벳: K, 감지된 알파벳: F
틀렸습니다! 현재 알파벳: F, 감지된 알파벳: K
틀렸습니다! 현재 알파벳: K, 감지된 알파벳: F
틀렸습니다! 현재 알파벳: F, 감지된 알파벳: K


In [15]:
import cv2
import mediapipe as mp
from ultralytics import YOLO

# YOLOv8 모델 로드
model = YOLO('asl_yolov8_model.pt')

# MediaPipe Hands 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 카메라 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

print("카메라가 실행되었습니다. 'q'를 눌러 종료하세요.")

# 문장 구성 변수
current_alphabet = None
sentence = []

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 가져올 수 없습니다.")
        break

    # 화면 좌우 반전 (거울 효과)
    frame = cv2.flip(frame, 1)

    # YOLOv8 예측
    results = model.predict(source=frame, conf=0.5, show=False, verbose=False)  # YOLOv8 예측 (로그 숨김)
    detections = results[0].boxes if results[0].boxes else []  # YOLOv8 탐지 결과

    # YOLO 탐지 결과 표시
    detected_alphabet = None
    for detection in detections:
        coords = detection.xyxy[0].tolist()  # 바운딩 박스 좌표
        x1, y1, x2, y2 = map(int, coords[:4])
        conf = detection.conf[0].item()
        if conf >= 0.6:  # 신뢰도 기준 추가
            class_id = int(detection.cls[0].item())  # 클래스 ID
            detected_alphabet = model.names[class_id]

            # YOLO 바운딩 박스 및 레이블 표시
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            label = f"{detected_alphabet} {conf:.2f}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # MediaPipe 손 랜드마크 감지
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    hand_results = hands.process(rgb_frame)

    # MediaPipe 랜드마크 표시
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
            )

    # 새 알파벳 감지 시 문장에 추가
    if detected_alphabet and detected_alphabet != current_alphabet:
        current_alphabet = detected_alphabet
        if detected_alphabet.lower() == "clear":  # 특정 단어로 문장 초기화
            sentence = []
            print("문장이 초기화되었습니다.")
        else:
            sentence.append(detected_alphabet)
            print(f"현재 문장: {''.join(sentence)}")

    # 화면 출력
    cv2.imshow("YOLOv8 + MediaPipe", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()
hands.close()



카메라가 실행되었습니다. 'q'를 눌러 종료하세요.
현재 문장: V
현재 문장: VF
현재 문장: VFW


## 오답노트

In [ ]:
import cv2
import mediapipe as mp
from ultralytics import YOLO
import time

# YOLOv8 모델 로드
model = YOLO('asl_yolov8_model.pt')

# MediaPipe Hands 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 유저로부터 원하는 알파벳 입력받기
target_alphabet = input("원하는 알파벳을 입력하세요: ").upper() # 대문자 처리 했고
print(f"원하는 알파벳: {target_alphabet}")

# 카메라 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

print("카메라가 실행되었습니다. 'q'를 눌러 종료하세요.")

last_wrong_time = 0  # 틀렸을 때의 타임스탬프
wrong_display_duration = 2  # 틀렸을 때 빨간색 상자를 표시할 시간(초)

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 가져올 수 없습니다.")
        break

    # 화면 좌우 반전 (거울 효과)
    frame = cv2.flip(frame, 1)

    # YOLOv8 예측
    results = model.predict(source=frame, conf=0.5, show=False, verbose=False)  # YOLOv8 예측 (로그 숨김)
    detections = results[0].boxes if results[0].boxes else []  # YOLOv8 탐지 결과

    detected_alphabet = None
    for detection in detections:
        coords = detection.xyxy[0].tolist()  # 바운딩 박스 좌표
        x1, y1, x2, y2 = map(int, coords[:4])
        conf = detection.conf[0].item()
        if conf >= 0.6:  # 신뢰도 기준 추가
            class_id = int(detection.cls[0].item())  # 클래스 ID
            detected_alphabet = model.names[class_id].upper()

            # 알파벳 비교 및 처리
            if detected_alphabet == target_alphabet:
                # 동일한 알파벳: 파란색 상자
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                label = f"Correct: {detected_alphabet}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            else:
                # 다른 알파벳: 빨간색 상자
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                label = f"Wrong: {detected_alphabet}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                last_wrong_time = time.time()  # 틀린 시간 기록

                # 틀린 메시지 출력
                print(f"틀렸습니다! 목표 알파벳: {target_alphabet}, 감지된 알파벳: {detected_alphabet}")

    # MediaPipe 손 랜드마크 감지
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    hand_results = hands.process(rgb_frame)

    # MediaPipe 랜드마크 표시
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
            )

    # # 틀렸을 때 빨간색 상자 지속 시간 처리
    # if time.time() - last_wrong_time < wrong_display_duration:
    #     cv2.putText(frame, "Wrong Input!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    # 화면 출력
    cv2.imshow("YOLOv8 + MediaPipe", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()
hands.close()


원하는 알파벳: L
카메라가 실행되었습니다. 'q'를 눌러 종료하세요.
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: K
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: K
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: L, 감지된 알파벳: V
